In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
data = pd.read_csv("https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/d546eaee765268bf2f487608c537c05e22e4b221/iris.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [4]:
data.species.value_counts()

virginica     50
setosa        50
versicolor    50
Name: species, dtype: int64

In [5]:
# Features and Label
features = data.iloc[:,:-1].values
label = data.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler
scFeatures = StandardScaler()
features = scFeatures.fit_transform(features)

In [8]:
#For Multiclass classification, Handling Label column is mandatory
# You need to use Dummy Variables

from sklearn.preprocessing import LabelEncoder
leSpecies = LabelEncoder()
label = leSpecies.fit_transform(label)

# To create Dummy Variables

oheLabels = tf.keras.utils.to_categorical(label)


In [10]:
# Create Train Test Split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(features,
                                                 oheLabels,
                                                 test_size=0.2,
                                                 random_state = 10)

In [11]:
#Modelling

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense( units = 12 , activation= "relu", input_shape=(4,)))
model.add(tf.keras.layers.Dense( units = 12 , activation= "relu"))
model.add(tf.keras.layers.Dense( units = 12 , activation= "relu"))
model.add(tf.keras.layers.Dense( units = 3 , activation= "softmax"))


In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
             loss="categorical_crossentropy",
             metrics=["accuracy"])

In [13]:
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self,cl):
        super(MyThresholdCallback,self).__init__()
        self.cl=cl
        
    def on_epoch_end(self, epoch, logs=None):
        testScore = logs['val_accuracy']
        trainScore = logs['accuracy']
        
        if testScore > trainScore and testScore >= self.cl:
            self.model.stop_training = True

In [14]:
# Fit model

model.fit(X_train, y_train, epochs=2000, validation_data=(X_test,y_test), callbacks=[MyThresholdCallback(cl=0.95)])

Epoch 1/2000
4/4 [==============================] - 3s 60ms/step - loss: 1.0614 - accuracy: 0.3500 - val_loss: 1.0001 - val_accuracy: 0.5667
Epoch 2/2000
4/4 [==============================] - 0s 9ms/step - loss: 0.8712 - accuracy: 0.6833 - val_loss: 0.8463 - val_accuracy: 0.6000
Epoch 3/2000
4/4 [==============================] - 0s 9ms/step - loss: 0.6923 - accuracy: 0.6917 - val_loss: 0.6822 - val_accuracy: 0.6333
Epoch 4/2000
4/4 [==============================] - 0s 9ms/step - loss: 0.5282 - accuracy: 0.7833 - val_loss: 0.5092 - val_accuracy: 0.7667
Epoch 5/2000
4/4 [==============================] - 0s 8ms/step - loss: 0.3924 - accuracy: 0.8583 - val_loss: 0.3780 - val_accuracy: 0.9333
Epoch 6/2000
4/4 [==============================] - 0s 8ms/step - loss: 0.3113 - accuracy: 0.9250 - val_loss: 0.2851 - val_accuracy: 1.0000


In [15]:
model.save('IrisModel')

INFO:tensorflow:Assets written to: IrisModel/assets


In [16]:
#Confusion Matrix for Entire dataset

from sklearn.metrics import confusion_matrix
y_true = label
y_pred = np.argmax(model.predict(features), axis = 1)
confusion_matrix(y_true,y_pred)

array([[50,  0,  0],
       [ 0, 46,  4],
       [ 0,  3, 47]])

In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       0.94      0.92      0.93        50
           2       0.92      0.94      0.93        50

    accuracy                           0.95       150
   macro avg       0.95      0.95      0.95       150
weighted avg       0.95      0.95      0.95       150



In [18]:
label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [22]:
# Model2

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(features,
                                                 label,
                                                 test_size=0.2,
                                                 random_state = 10)

model2 = tf.keras.models.Sequential()

model2.add(tf.keras.layers.Dense( units = 12 , activation= "relu", input_shape=(4,)))
model2.add(tf.keras.layers.Dense( units = 12 , activation= "relu"))
model2.add(tf.keras.layers.Dense( units = 12 , activation= "relu"))
model2.add(tf.keras.layers.Dense( units = 3 , activation= "softmax"))

model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
             loss="sparse_categorical_crossentropy", #No need to create dummy variables :)
             metrics=["accuracy"])



model2.fit(X_train, y_train, epochs=2000, validation_data=(X_test,y_test), callbacks=[MyThresholdCallback(cl=0.95)])

Epoch 1/2000
4/4 [==============================] - 0s 42ms/step - loss: 0.8710 - accuracy: 0.6083 - val_loss: 0.8212 - val_accuracy: 0.5667
Epoch 2/2000
4/4 [==============================] - 0s 7ms/step - loss: 0.6835 - accuracy: 0.7000 - val_loss: 0.6407 - val_accuracy: 0.6667
Epoch 3/2000
4/4 [==============================] - 0s 13ms/step - loss: 0.5329 - accuracy: 0.7500 - val_loss: 0.4867 - val_accuracy: 0.7667
Epoch 4/2000
4/4 [==============================] - 0s 7ms/step - loss: 0.4026 - accuracy: 0.8333 - val_loss: 0.3815 - val_accuracy: 0.8333
Epoch 5/2000
4/4 [==============================] - 0s 7ms/step - loss: 0.3194 - accuracy: 0.8750 - val_loss: 0.2966 - val_accuracy: 0.9000
Epoch 6/2000
4/4 [==============================] - 0s 8ms/step - loss: 0.2743 - accuracy: 0.9000 - val_loss: 0.2402 - val_accuracy: 0.9667
